In [40]:
#| default_exp get_examples

%pip install -q langchain_core
%pip install -q langchain_openai
%pip install -q psycopg2-binary
%pip install -q nbdev


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [41]:

from langchain_core.prompts import PromptTemplate

In [42]:
getExamplesPrompt = PromptTemplate.from_template("""Return one ore more json objects together in an array to describe the jobs in a job posting. If there is only one job posting presented here, return a json object containing a summary of every separate piece of information in the job posting. If the job describes multiple roles, create separate json objects for each role. Here is the job posting:\n{job_posting}""")



1. Get last month's  job listings

In [44]:
import json

# Opening JSON file
f = open('../posts.json')

# returns JSON object as 
# a dictionary
posts = json.load(f)

2. Create key/value pairs from each post
* update the keys as we go

In [48]:
from langchain_openai import ChatOpenAI
import  json

model = ChatOpenAI()

parsed_data = [  ]
result = {}

# Start with some keys we are sure will be needed
keys = [
    'job_title',
    'company',
    'location',
    'employment_type',
    'tech_stack',
    'job_requirements',
    'salary',
]

postItems = posts[:2]

for i in range(len(postItems)):
        msg = getExamplesPrompt.format(job_posting = postItems[i]['comment'], key_values = ", ".join(keys))
        
        resp = model.invoke(msg)
        parsed_datum = json.loads(resp.content)
        
        for  job in  parsed_datum:
            parsed_data.append(job)

            parsed_data_keys = list(job.keys()) 
    
            for key in parsed_data_keys:
                if key not in keys:
                    keys.append(key)

            # We add keys here, so subsequent examples can use them
            keyStr = ", ".join(keys)

        print(parsed_data)


You will be returning one ore more json objects together in an array to describe the jobs in a job posting. Here are examples of the format of the json object you should return:

Here are the key values currently being used to categorize jobs:
.
 If there is only one job posting presented here, return a json object containing a summary of every separate piece of information in the job posting. If the job describes multiple roles, create a separate json object for each role. Use the existing key values where they are relevant. If there is no relevant existing key, add new, relevant key to your json object:
Tandem Health | Software Engineers | On-site in Stockholm, Sweden | Full time
At Tandem Health we're building a clinician copilot to allow clinicians to focus on care rather than administration. We’re building, launching and iterating quickly, already at $300k ARR (+35% MoM) with over 50 care providers using the product every day. With our target set at the ginormous European healthca

In [53]:

# let's see how frequently each key appears
tally = {}
result = {}
for key in keys:
    result[key] = [datum[key] for datum in parsed_data]
    tally[key] = sum([1 for datum in parsed_data if key in datum])

result, tally

Save the keys for use, and human editing


In [54]:
from hn_jobs_chat.util import saveJSON

saveJSON('keys.json', keys)
saveJSON('parsed_data-july.json', parsed_data)

[[{'job_title': 'Senior Software Engineer',
   'company': 'Irth Solutions',
   'description': 'At Irth Solutions we build software that improve resilience and reduce risk to critical infrastructure that millions of people and businesses rely on each day. Our systems handle millions of requests a day, coordinating activities between critical infrastructure providers, contract locators, excavators, and a variety of other stakeholders throughout North America.',
   'technical_requirements': ['C# development experience using .NET Core, WebAPI, and Entity Framework',
    'Experience building browser applications using Angular and Typescript',
    'Strong relational database experience with Postgres (preferred) or MS SQL Server including SQL, database functions, indexing, and performance tuning',
    'Kubernetes, Docker, Linux'],
   'apply_link': 'https://apply.workable.com/irth-solutions/j/33AD66BA3A/'}],
 {'company': 'Tandem Health',
  'job_title': 'Software Engineer',
  'location': 'Stock

In [1]:
#| hide
from nbdev import nbdev_export
nbdev_export()

/Users/Bumpant/Development/LLM/HN-search/.venv/lib/python3.12/site-packages/nbdev/doclinks.py:101: SyntaxWarning: invalid escape sequence '\.'
  for file in globtastic(dest, file_glob="*.py", skip_file_re='^_', skip_folder_re="\.ipynb_checkpoints"):


JSONDecodeError: Expecting value: line 1 column 1 (char 0)